<a href="https://colab.research.google.com/github/AlexeyTri/Home_work_NN/blob/main/HomeWork_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.metrics import r2_score, log_loss
from sklearn.model_selection import cross_val_predict, KFold, train_test_split
import matplotlib.pyplot as plt
from collections.abc import Sequence
from sklearn.impute import SimpleImputer

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/AlexeyTri/Home_work_NN/refs/heads/main/data/gbm-data.csv")
df_array = df.values

In [3]:
df['Activity'].value_counts()

,count
Activity,
1,2034
0,1717


In [4]:
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer = imputer.fit(df)

In [5]:
X = df.iloc[:, 1:]
y = df.iloc[:, 0]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=241)

In [7]:
def gradient_boost_classifier(X: Sequence[pd.DataFrame, np.array], y: Sequence[pd.DataFrame, np.array], learning_rate:  np.array, n_estimators: int, verbose: bool, random_state: int):

    log_loss_fun = []

    for lr in learning_rate:

        gbc = GradientBoostingClassifier(learning_rate=lr, n_estimators=n_estimators, verbose=verbose, random_state=random_state)
        gbc.fit(X, y)
        pred = gbc.staged_decision_function(X)

        for i, value in  enumerate(pred):
            y_pred_sigm = 1 / (1 + np.exp(- value))
            log_loss_ = log_loss(y, y_pred_sigm) # change X
            log_loss_fun.append([lr, i, round(log_loss_, 2)])

    return log_loss_fun

In [8]:
logloss_train = gradient_boost_classifier(X_train, y_train, learning_rate=[1, 0.5, 0.3, 0.2, 0.1], n_estimators=250, verbose=True, random_state=241)
logloss_test = gradient_boost_classifier(X_test, y_test, learning_rate=[1, 0.5, 0.3, 0.2, 0.1], n_estimators=250, verbose=True, random_state=241)

      Iter       Train Loss   Remaining Time 
         1           1.0270            2.05m
         2           0.9644            1.75m
         3           0.9325            1.70m
         4           0.9000            1.57m
         5           0.8660            1.57m
         6           0.8448            1.51m
         7           0.8272            1.40m
         8           0.8124            1.32m
         9           0.8022            1.26m
        10           0.7905            1.21m
        20           0.6253           59.45s
        30           0.5217            1.05m
        40           0.4362            1.24m
        50           0.3711            1.22m
        60           0.3055            1.23m
        70           0.2474            1.12m
        80           0.2092            1.00m
        90           0.1700           54.04s
       100           0.1460           48.75s
       200           0.0347           13.90s
      Iter       Train Loss   Remaining Time 
        

In [9]:
logloss_train_pd = pd.DataFrame(logloss_train,columns=['lr', 'iter', 'score_loss'])
logloss_test_pd = pd.DataFrame(logloss_test,columns=['lr', 'iter', 'score_loss'])

In [10]:
logloss_train_pd[logloss_train_pd['lr'] == 0.2].sort_values('score_loss', ).iloc[0, 1:]
logloss_test_pd[logloss_test_pd['lr'] == 0.2].sort_values('score_loss', ).iloc[0, 1:]

,982
iter,232.00
score_loss,0.02


In [ ]:
rfc = RandomForestClassifier(n_estimators=232, verbose=True, random_state=241)
rfc.fit(X_train, y_train)
log_loss(y_test, rfc.predict_proba(X_test))

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    1.2s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    4.9s


In [12]:
gbc = GradientBoostingClassifier(n_estimators=232,learning_rate=0.2, verbose=True, random_state=241)
gbc.fit(X_train, y_train)
log_loss(y_test, rfc.predict_proba(X_test))

      Iter       Train Loss   Remaining Time 
         1           1.2599           50.78s
         2           1.1801           49.19s
         3           1.1231           48.72s
         4           1.0833           48.18s
         5           1.0516           47.74s
         6           1.0273           47.69s
         7           1.0062           47.35s
         8           0.9885           47.00s
         9           0.9708           46.82s
        10           0.9566           46.40s
        20           0.8637           44.70s
        30           0.8027           47.43s
        40           0.7553           44.91s
        50           0.7094           41.38s
        60           0.6676           38.45s
        70           0.6383           35.74s
        80           0.6119           33.32s
        90           0.5787           31.86s
       100           0.5541           30.55s
       200           0.3618            7.15s


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 232 out of 232 | elapsed:    0.1s finished


0.45581642866472283